# TIMESERIES ANALYSIS

# Business Understanding

This project focuses on the application of time series regression analysis to forecast sales for Corporation Favorita, a prominent grocery retailer based in Ecuador.

The primary objective is to develop a robust model capable of accurately forecasting future sales by leveraging the extensive time series data of thousands of products sold across various Favorita locations. The resulting forecasts will provide valuable insights to the store's management, enabling them to formulate effective inventory and sales plans.

Through this research, we will construct models based on historical analysis, establish scientific hypothesis using time-stamped historical data, and employ these models to observe patterns and guide strategic decision-making in the future. By delving into the data, our aim is to optimize operations and ultimately drive sales growth for Favorita Corporation, supporting the management team in extracting meaningful insights from their vast dataset.

# Hypothesis

Null Hypothesis: Sales are not affected by promotion, oil prices and holidays.

Alternate Hypothesis: Sales are affected by promotion, oil prices and holidays.

# Analytical Questions

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

In [43]:
# Importing the needed packages
import pandas as pd
import numpy as np

# Libraries to create connection string to SQL server
import pyodbc
from dotenv import dotenv_values

# Libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Library for imputing missing values
from sklearn.impute import SimpleImputer

# Library for modelling
from sklearn.linear_model import LinearRegression

# Library for working with operating system
import os

# Library to handle warnings
import warnings
warnings.filterwarnings('ignore')

In [44]:
# Set Matplotlib defaults
plt.style.use("seaborn-v0_8")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

# Data Understanding

### Accessing and loading the datasets

The first dataset was collected from a SQL database by first passing a connection string using the pyodbc library. Afterwards a SQL query was used to obtain the dataset. This is as shown below.

In [45]:
# Load the environment variable in the .env file into a dictionary

environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the .env file
server = environment_variables.get('SERVER')
database = environment_variables.get('DATABASE')
username = environment_variables.get('USERNAME')
password = environment_variables.get('PASSWORD')

# The connection string is an f string that includes all the variable above to establish a connection to the server.
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

In [46]:
# Use the connect method of the pyodbc library to pass in the connection string.
# Check your internet connection if it takes more time than necessary.

connection = pyodbc.connect(connection_string)

# Get the oil dataset using the SQL query shown below
query1 = 'Select * from dbo.oil'
oil = pd.read_sql(query1, connection)

# Get the holiday dataset using the SQL query shown below
query2 = 'Select * from dbo.holidays_events'
holiday = pd.read_sql(query2, connection)

# Get the stores dataset using the SQL query shown below
query3 = 'Select * from dbo.stores'
stores = pd.read_sql(query3, connection)

# Save the datasets
oil.to_csv(r'oil.csv')
holiday.to_csv(r'holiday.csv')
stores.to_csv(r'stores.csv')

In [ ]:
# Loading the other datasets

sample_submission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
transactions = pd.read_csv('transactions.csv')

### EDA

In [ ]:
# Evaluating the first five rows of the holiday dataset

holiday.head()

In [ ]:
# Checking for missing values and the datatypes of the columns in the holiday dataset

holiday.info()

In [ ]:
# Checking for duplicates on the holiday dataset

holiday.duplicated().sum()

In [ ]:
# Evaluating the first five rows of the oil dataset

oil.head()

In [ ]:
# Checking for missing values and the datatypes of the columns in the oil dataset

oil.info()

In [ ]:
# Counting the missing values in the oil datast

oil.isna().sum()

In [ ]:
# Checking for duplicates on the oil dataset

oil.duplicated().sum()

In [ ]:
# Evaluating the first five rows of the stores dataset

stores.head()

In [ ]:
# Checking for missing values and the datatypes of the columns in the holiday dataset

stores.info()

In [ ]:
# Checking for duplicates on the stores dataset

stores.duplicated().sum()

In [ ]:
# Evaluating the first five rows of the sample_submission dataset

sample_submission.head()

In [ ]:
# Checking for missing values and the datatypes of the columns in the sample_submission dataset

sample_submission.info()

In [ ]:
# Checking for duplicates on the sample_submission dataset

sample_submission.duplicated().sum()

In [ ]:
# Evaluating the first five rows of the test dataset

test.head()

In [ ]:
# Checking for missing values and the datatypes of the columns in the test dataset

test.info()

In [ ]:
# Checking for duplicates on the test dataset

test.duplicated().sum()

In [ ]:
# Evaluating the first five rows of the train dataset

train.head()

In [ ]:
# Checking the datatypes of the columns in the train dataset

train.info()

In [ ]:
# Checking for missing values in the train dataset

train.isna().sum()

In [ ]:
# Checking for duplicates on the train dataset

train.duplicated().sum()

In [ ]:
# Evaluating the first five rows of the transactions dataset

transactions.head()

In [ ]:
# Checking for missing values and the datatypes of the columns in the transactions dataset

transactions.info()

In [ ]:
# Checking for duplicates on the transactions dataset

transactions.duplicated().sum()

# Problems Identified

The oil dataset has 43 missing values on the 'dcoilwtico' column.

For all the datasets that have a 'date' column, the 'date' column is present as object datatype instead of datetime datatype.

The datasets are seperate, and need to be merged together for better analysis.

# Data Preparation

The problems identified with the datasets will be handled to prepare the data for analysis and modelling.

### Filling missing values

In [ ]:
# create an instance of the SimpleImputer class with mean strategy
imputer = SimpleImputer(strategy='mean')

# fit the imputer to the 'dcoilwtico' column of oil dataset
imputer.fit(oil[['dcoilwtico']])

# Impute missing values on the 'dcoilwtico' column of oil dataset with the imputer
oil['dcoilwtico'] = imputer.transform(oil[['dcoilwtico']])

oil.head()

### Combine the datasets based on common columns

In [ ]:
# Merge transactions dataset to train on 'date' and 'store_nbr' columns
df1 = pd.merge(train, transactions, on=['date', 'store_nbr'])

# Merge holiday dataset to df1 on 'date' column
df2 = pd.merge(df1, holiday, on='date')

# Merge oil dataset to df2 on 'date' column
df3 = pd.merge(df2, oil, on='date')
df3

# Merge store dataset to df3 on 'store_nbr' column
df4 = pd.merge(df3, stores, on='store_nbr')

df4.head()

### Change the datatype of the 'date' column from object to datetime

In [ ]:
df4['date'] = pd.to_datetime(df4['date'])

df4.info()

The missing values in the 'dcoilwtico' column of the oil dataset have been filled, the datasets have been merged and the datatype of the 'date' column has been changed from object to datetime.

In [ ]:
# Evaluating 'type_x' column on df4

df4['type_x'].unique()

In [ ]:
# Evaluating 'type_y' column on df4

df4['type_y'].unique()

In [ ]:
# Rename 'type_x' and type_y' to 'holiday_type' and 'store_type' respectively

df4 = df4.rename(columns={'type_x': 'holiday_type', 'type_y': 'store_type'})
df4.head()

In [ ]:
# Rename df4 to train_data and save the DataFrame

train_merged = df4
train_merged.to_csv(r'train_merged.csv')

In [ ]:
# Generate summary statistics for numerical columns in train_data DataFrame

train_merged.describe()

In [ ]:
# Create a boxplot of the 'transactions' column grouped by 'locale'
sns.boxplot(x='transactions', y='locale', data=train_merged)

# Show the plot
plt.show()

In [ ]:
# Create a boxplot of transactions by City
sns.barplot(x='transactions', y='city', data=train_merged)

width=0.5,  # Adjust the width of the boxes
fliersize=3, # Adjust the size of the outliers
showmeans=True, # Show the mean value
meanline=True, # Show a line for the mean
notch=True, # Make the boxes "notched"

# Add a title and labels for the x and y axis
plt.title('Transactions by City', fontsize=18)
plt.xlabel('Frequency', fontsize=16)
plt.ylabel('City', fontsize=16)

# Show the plot
plt.show()

In [ ]:
# Create a histogram of the 'transactions' column
train_merged.transactions.hist()

# Add labels to the x-axis, y-axis, and title
plt.xlabel('Transactions', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
plt.title('Histogram of Transactions', fontsize=20)

# Show the plot
plt.show()

In [ ]:
# Create a DataFrame of train_data with numerical columns only
train_merged_num = train_merged.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix of the numerical columns
corr_matrix = train_merged_num.corr()

# Visualizing the correlation matrix with a heatmap
sns.heatmap(corr_matrix, annot=True)

# Save the chart as an image file
# plt.savefig('Correlation of the numerical columns of the train dataset.png')

In [ ]:
# Create new features from the 'date' column

train_merged['year'] = train_merged.date.dt.year
train_merged['month'] = train_merged.date.dt.month
train_merged['dayofmonth'] = train_merged.date.dt.day
train_merged['dayofweek'] = train_merged.date.dt.dayofweek
train_merged['dayname'] = train_merged.date.dt.strftime('%A')

train_merged.head()

In [ ]:
# Display random sample of 8 rows

train_merged.sample(8, random_state = 42)

# Hypothesis Testing

# Answering Analytical Questions